In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
from deep_tools import f
from deep_tools import DataGenerator

In [132]:
register=pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
launch=pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
create=pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
activity=pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])

In [133]:
register.head()

,user_id,register_day,register_type,device_type
0,744025,1,1,283
1,1270299,1,1,259
2,571220,1,1,2
3,1308501,1,0,23
4,745554,1,2,0


![title](./img/1.png)

In [135]:
launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


![title](./img/2.png)

In [137]:
create.head()

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


![title](./img/3.png)

In [139]:
activity.head()

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


![title](./img/4.png)

计算序列长度，持续时间=数据总时间-注册时间

In [142]:
register['seq_length']=31-register['register_day']
register.head()

,user_id,register_day,register_type,device_type,seq_length
0,744025,1,1,283,30
1,1270299,1,1,259,30
2,571220,1,1,2,30
3,1308501,1,0,23,30
4,745554,1,2,0,30


构建字典存储用户在持续时间内，不同日期的数据

In [144]:
user_queue={i:[] for i in range(1,31)}

In [145]:
for index,row in register.iterrows():
    user_queue[row[-1]].append(row[0])#row[-1]是seq_length,row[0]是user_id

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/3570424115.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  user_queue[row[-1]].append(row[0])#row[-1]是seq_length,row[0]是user_id


key表示序列长度（持续天数），value表示用户

In [147]:
user_queue

{1: [np.int64(355948),
  np.int64(1141243),
  np.int64(885314),
  np.int64(473037),
  np.int64(1161976),
  np.int64(1105249),
  np.int64(1034711),
  np.int64(444663),
  np.int64(629789),
  np.int64(684292),
  np.int64(1220188),
  np.int64(38479),
  np.int64(1166260),
  np.int64(739798),
  np.int64(1023596),
  np.int64(731282),
  np.int64(759110),
  np.int64(327708),
  np.int64(1102438),
  np.int64(466836),
  np.int64(496107),
  np.int64(1222898),
  np.int64(166312),
  np.int64(236360),
  np.int64(473209),
  np.int64(91356),
  np.int64(393059),
  np.int64(439146),
  np.int64(956525),
  np.int64(49020),
  np.int64(1306827),
  np.int64(1198470),
  np.int64(857247),
  np.int64(316571),
  np.int64(136844),
  np.int64(691785),
  np.int64(1272376),
  np.int64(265670),
  np.int64(560000),
  np.int64(757319),
  np.int64(1133544),
  np.int64(822853),
  np.int64(979976),
  np.int64(664211),
  np.int64(480676),
  np.int64(277894),
  np.int64(919793),
  np.int64(107804),
  np.int64(1191738),
  np.i

In [148]:
class user_seq:
    
    def __init__(self,register_day,seq_length,n_features):
        self.register_day=register_day
        self.seq_length=seq_length
        self.array=np.zeros([self.seq_length,n_features]) #构建矩阵：持续天数*特征个数，后续新创建的特征来往里面填充
        self.array[0,0]=1
        self.page_rank=np.zeros([self.seq_length])
        self.pointer=1
        
    def put_feature(self,feature_number,string):
        for i in string.split(','):
            pos,value=i.split(':') #注册后第几天进行了登录，1为指示符
            self.array[int(pos)-self.register_day,feature_number]=1

    def put_PR(self,string):
        for i in string.split(','):
            pos,value=i.split(':')
            self.page_rank[int(pos)-self.register_day]=value

    def get_array(self):
        return self.array
    
    def get_label(self):
        self.label=np.array([None]*self.seq_length)
        active=self.array[:,:10].sum(axis=1)
        for i in range(self.seq_length-7):
            self.label[i]=1*(np.sum(active[i+1:i+8])>0)
        return self.label

In [149]:
n_features=12
data={row[0]:user_seq(register_day=row[1],seq_length=row[-1],n_features=n_features) for index,row in register.iterrows()}

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/804871407.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data={row[0]:user_seq(register_day=row[1],seq_length=row[-1],n_features=n_features) for index,row in register.iterrows()}


得到每个用户特征列表

In [151]:
data

{np.int64(744025): <__main__.user_seq at 0x16b78e440>,
 np.int64(1270299): <__main__.user_seq at 0x16b78ef80>,
 np.int64(571220): <__main__.user_seq at 0x16b77cf40>,
 np.int64(1308501): <__main__.user_seq at 0x16b78ea40>,
 np.int64(745554): <__main__.user_seq at 0x16e1dd480>,
 np.int64(1031012): <__main__.user_seq at 0x16e1dd600>,
 np.int64(913297): <__main__.user_seq at 0x16e1df940>,
 np.int64(266500): <__main__.user_seq at 0x16e1dcd00>,
 np.int64(475120): <__main__.user_seq at 0x16e1debc0>,
 np.int64(547944): <__main__.user_seq at 0x16e1dca00>,
 np.int64(916655): <__main__.user_seq at 0x16e1ddfc0>,
 np.int64(719262): <__main__.user_seq at 0x16e1ddd80>,
 np.int64(1026175): <__main__.user_seq at 0x16e1dd840>,
 np.int64(1140342): <__main__.user_seq at 0x16e1dee60>,
 np.int64(688100): <__main__.user_seq at 0x16e1dd2a0>,
 np.int64(1342459): <__main__.user_seq at 0x16e1dfdc0>,
 np.int64(926263): <__main__.user_seq at 0x16e1df040>,
 np.int64(40710): <__main__.user_seq at 0x15dcbdba0>,
 np.i

#### 登录信息

In [153]:
launch['launch']=1 #每个用户每天登录次数
launch_table=launch.groupby(['user_id','launch_day'],as_index=False).agg({'launch':'sum'})
launch_table.head()

,user_id,launch_day,launch
0,16,13,1
1,16,14,1
2,16,15,1
3,16,18,1
4,16,19,1


In [154]:
def record_to_sequence(table): #得到用户特征序列表
    table.columns=['user_id','day','value']
    table.sort_values(by=['user_id','day'],inplace=True)
    table['string']=table.day.map(str)+':'+table.value.map(str)
    table=table.groupby(['user_id'],as_index=False).agg({'string':lambda x:','.join(x)})
    return table

In [155]:
launch_table=record_to_sequence(launch_table) #序列特征结果
launch_table.head()

,user_id,string
0,16,"13:1,14:1,15:1,18:1,19:1,20:1,21:1,22:1,23:1"
1,30,24:1
2,98,16:1
3,105,"12:1,14:1,15:1,16:1,17:1,18:1,19:1,20:1,21:1,2..."
4,176,"27:1,28:1,29:1,30:1"


例如ID=16的用户会在其特征表的指定位置的（13,14,15,18...行进行填充）

In [157]:
for index,row in launch_table.iterrows(): #根据登录信息对用户特征表进行填充
    data[row[0]].put_feature(1,row[1]) #在指定特征位置上进行填充

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/905411779.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(1,row[1]) #在指定特征位置上进行填充


#### 创作视频信息

In [159]:
create['create']=1
create_table=create.groupby(['user_id','create_day'],as_index=False).agg({'create':'sum'})
create_table=record_to_sequence(create_table)
for index,row in create_table.iterrows():
    data[row[0]].put_feature(2,row[1])

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/3675570800.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(2,row[1])


#### 用户使用时行为特征，例如点赞，转发等


分别对不同行为进行统计，构建6种不同行为特征

In [162]:
for i in range(6):
    act=activity[activity.act_type==i].copy()
    act=act.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
    act=record_to_sequence(act)
    for index,row in act.iterrows():
        data[row[0]].put_feature(i+3,row[1])

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/2709730670.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(i+3,row[1])
/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/2709730670.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(i+3,row[1])
/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/2709730670.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

#### 产生行为的界面信息

In [164]:
for i in range(1): #暂不作为特征
    act=activity[activity.page==i].copy()
    act=act.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
    act=record_to_sequence(act)
    for index,row in act.iterrows():
        data[row[0]].put_feature(i+9,row[1]) 

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/545838185.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(i+9,row[1])


#### 观看其他用户作品信息

In [166]:
watched=register.loc[:,['user_id']].copy()
watched.columns=['author_id']
watched=pd.merge(watched,activity[activity.author_id!=activity.user_id],how='inner') #只得到交集，相当于看别人视频的
watched=watched.groupby(['author_id','act_day'],as_index=False).agg({'video_id':'count'})
watched=record_to_sequence(watched)
for index,row in watched.iterrows():
    data[row[0]].put_feature(10,row[1])

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/3479833359.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(10,row[1])


#### 观看自己的作品信息

In [168]:
watched=activity[activity.author_id==activity.user_id].copy()
watched=watched.groupby(['user_id','act_day'],as_index=False).agg({'video_id':'count'})
watched=record_to_sequence(watched)
for index,row in watched.iterrows():
    data[row[0]].put_feature(11,row[1]) 

/var/folders/1r/x304lc4s2dq9stsg9pmdqbtr0000gn/T/ipykernel_5995/1063660583.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[row[0]].put_feature(11,row[1])


#### 制作数据标签

活跃用户定义为：在未来7天内使用过APP（在上述任一类型日志中出现过）

对用户从注册开始时进行统计，对于每1天的数据展开，如果其7天后仍有行为产生，则标签为1

In [172]:
label={user_id:user.get_label() for user_id,user in data.items()} 

In [173]:
label

{np.int64(744025): array([np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1),
        np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), None, None, None, None,
        None, None, None], dtype=object),
 np.int64(1270299): array([np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), None, None, None, None,
        None, None, None], dtype=object),
 np.int64(571220): array([np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0),
        np.int64(0), np.int64(0), np.int64(0), np.int64(0), np

#### 用户特征数据即为之前提取的data中的各项特征，转换成ndarray即可

In [175]:
data={user_id:user.get_array() for user_id,user in data.items()}

#### 合并上述提取方法

In [177]:
register=pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
launch=pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
create=pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
activity=pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])

data_generator=DataGenerator(register,launch,create,activity)

/Users/haruko/AnacondaProjects/DataScienceCode/路飞学城数据分析数据挖掘/数据挖掘竞赛-优胜解决方案/第一章：快手用户活跃度预测/快手用户活跃度/deep_tools.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  #初始化self.data
/Users/haruko/AnacondaProjects/DataScienceCode/路飞学城数据分析数据挖掘/数据挖掘竞赛-优胜解决方案/第一章：快手用户活跃度预测/快手用户活跃度/deep_tools.py:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  
/Users/haruko/AnacondaProjects/DataScienceCode/路飞学城数据分析数据挖掘/数据挖掘竞赛-优胜解决方案/第一章：快手用户活跃度预测/快手用户活跃度/deep_tools.py:82: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behav

构建RNN网络模型

In [179]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers

# 1. 禁用eager execution
tf.compat.v1.disable_eager_execution()

# 2. 设置超参数
n_features = 12  # 特征维度
n_hu = 32      # GRU隐藏单元数量

# 3. 构建模型
with tf.compat.v1.variable_scope('train'):
    # 变量与输入
    lr = tf.compat.v1.placeholder(tf.float32, [], name='learning_rate')
    W_out = tf.compat.v1.get_variable('W_out', [n_hu, 1])
    b_out = tf.compat.v1.get_variable('b_out', [1])
    
    x = tf.compat.v1.placeholder(tf.float32, [None, None, n_features])
    y = tf.compat.v1.placeholder(tf.float32, [None, None])
    
    batch_size = tf.shape(x)[0]
    seq_length = tf.shape(x)[1]
    
    # RNN层 - 使用Keras的GRU层
    gru_layer = layers.GRU(n_hu, return_sequences=True)
    outputs = gru_layer(x)
    
    # 输出层
    outputs_reshaped = tf.reshape(outputs, [-1, n_hu])
    logits = tf.matmul(outputs_reshaped, W_out) + b_out
    logits = tf.reshape(logits, tf.stack([batch_size, seq_length]))

In [180]:
#选择部分预测结果与标签当做训练损失计算
logits_local_train=logits[:,:-14]
label_local_train=y[:,:-14]

In [181]:
# 设置损失函数
# 在 TF2 中使用 keras.regularizers 替代 contrib.layers
regularizer = tf.keras.regularizers.l2(l2=0.00001)

# 计算 L2 正则化惩罚项
penalty = 0
for var in tf.compat.v1.trainable_variables():
    penalty += regularizer(var)

# 使用 tf.nn.sigmoid_cross_entropy_with_logits 替代 tf.losses
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=label_local_train,
    logits=logits_local_train
)
obj_local = tf.reduce_mean(cross_entropy) + penalty

# 优化器
optimizer = tf.compat.v1.train.AdamOptimizer(lr)
step_local = optimizer.minimize(obj_local)

# 选择部分预测结果与标签当做测试损失计算
logits_local_test = logits[:,-8]
label_local_test = y[:,-8]

In [182]:
def train(n_obs=1000,step=1000,lr_feed=0.01):

    date_seq=[31]+list(range(2,16))+[16]*15
    variables=[step_local,obj_local,label_local_train,logits_local_train]
    
    for i in range(step):
        length,id_list,data_x,data_y=data_generator.next_batch(n_obs)
        _,los,lab,log=sess.run(variables,
                               feed_dict={x:data_x,
                                          y:data_y,
                                          lr:lr_feed})

In [183]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 设置日志级别

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [184]:
train(n_obs=1000,step=2000,lr_feed=0.01)

In [185]:
def test():
    
    n_NA=14
    date_seq=[31]+list(range(2,16))+[16]*15
    variables_1=[obj_local,logits_local_train,label_local_train]
    variables_2=[logits_local_test,label_local_test]
        
    obs_count,cum_loss,correct=0,0,0
    user,prob,real=[],[],[]

    #训练损失
    for length,id_list,data_x,data_y in zip(*data_generator.get_set('train')):
        _obj,_logits_train,_label_train=sess.run(variables_1,
                           feed_dict={x:data_x,
                                      y:data_y,
                                      lr:0.001})
        obs_count+=(length-n_NA)*len(id_list)
        cum_loss+=_obj*(length-n_NA)*len(id_list)
        correct+=np.sum((1*(_logits_train>0)==_label_train))

    #测试损失
    for length,id_list,data_x,data_y in zip(*data_generator.get_set('test')):
        _=sess.run(variables_2,
                           feed_dict={x:data_x,
                                      y:data_y,
                                      lr:0.001})
        
        _logits_test,_label_test=_
        real+=list(_label_test)

        user+=list(id_list)
        prob+=list(1/(1+np.exp(-_logits_test.reshape([-1]))))
    
    #训练损失
    print('train_loss',cum_loss/obs_count)
    
    #测试损失
    result=pd.DataFrame({'user_id':user,'prob':prob,'label':real})
    print('test_score:',f(result))
    
    return result

### 评估标准：

![title](./img/5.png)

In [188]:
test()

train_loss 0.4664805
test_score: [np.float64(0.8046650279797081), np.float64(0.8051034337918723), np.float64(0.8044052398056202), np.float64(0.804049935063214), np.float64(0.8039667649423746), np.float64(0.802861599698779)]


,user_id,prob,label
0,1274576,0.858481,1.0
1,109973,0.840304,1.0
2,134299,0.840304,0.0
3,1005835,0.613618,1.0
4,864582,0.855402,1.0
...,...,...,...
37441,621946,0.055638,0.0
37442,825398,0.974597,1.0
37443,1307296,0.615211,1.0
37444,39384,0.222716,0.0
